# Generate Statistic and TIFFs for Site Location Analysis

In [1]:
import os
import json
import numpy as np
import pandas as pd

import initialise
import common
from analysis_utils import calc_statistics

In [2]:
if common.GDAL_DATA is not None:
    os.environ['GDAL_DATA'] = common.GDAL_DATA
import gdal
from gdal import osr

In [3]:
SCENARIOS = ['within-site', 'out-of-site']
DISPLAY_NAMES = [' '.join([s.capitalize(), 'Models']) for s in SCENARIOS]
MODELS_DIR = [
    os.path.join(common.MODELS_DIR, f'{SCENARIOS[0]}_models'),
    os.path.join(common.MODELS_DIR, f'{SCENARIOS[1]}_models'),
    os.path.join(common.MODELS_DIR, 'comparison_models', 'test2'),
    os.path.join(common.MODELS_DIR, 'comparison_models', 'test3'),
]
OUTPUT_DIR = os.path.join(common.MAPS_DIR, 'Gridded_sites')
NODATA = common.GDAL_NODATA_VALUE
BANDS = ['RMSE', 'Bias']

In [4]:
all_samples = pd.read_csv(os.path.join(common.DATASETS_DIR, 'samples_365days.csv'), index_col=0)
models = [
    {'type': 'Multi-tempCNN', 'preds_file': f'ensemble{common.ENSEMBLE_SIZE}_{common.ANALYSIS_MODEL}.csv'},
    {'type': 'Modis-tempCNN', 'preds_file': f'predictions_{common.MODIS_TEMPCNN_MODEL}.csv'},
]
ifile_names = [m['preds_file'] for m in models for _ in range(len(SCENARIOS))]
ofile_names = [f'{mt["type"]}_{sc}_sites.tif' for mt in models for sc in SCENARIOS]

### Calculate the statistics for each grid cell

In [5]:
def gen_gridded_data(predictions, samples, y):
    predict = predictions.merge(all_samples[['Latitude', 'Longitude', y]], left_index=True, right_index=True)
    predict['Latitude'] = np.ceil((predict.Latitude * 2))/2
    predict['Longitude'] = np.floor((predict.Longitude * 2))/2

    counts = predict.groupby(['Latitude', 'Longitude']).size()
    locations = []
    locs = predict.set_index(['Latitude', 'Longitude']).loc[counts[counts >= 10].index].groupby(['Latitude', 'Longitude'])
    for col in predict.columns[:-3]:
        l = locs.apply(lambda x: calc_statistics(x[y], x[col])) 
        locations.append(pd.DataFrame(list(l.values), index=l.index))
    locations = pd.concat(locations).groupby(level=[0,1]).mean()
    locations['NumSamples'] = counts
    return locations

### Summary statistics

In [6]:
def display_summary(locations):
    print('Proportion of grid cells with RMSE < 20:', np.round(locations.RMSE.lt(20).sum() / locations.shape[0], 2), '\n')
    display(pd.DataFrame([locations.min(), locations.median(), locations.mean(), locations.max(), locations.std()],
                         index=['min', 'median', 'mean', 'max', 'std dev']).round(2))

### Region summary

In [7]:
def display_region(locations, long_range, lat_range):
    temp_df = locations.reset_index()
    temp_df = temp_df[temp_df.Latitude.between(lat_range[0], lat_range[1]) & temp_df.Longitude.between(long_range[0], long_range[1])]
    print('RMSE weighted average:', np.round(np.sqrt((temp_df.RMSE ** 2 * temp_df.NumSamples).sum() / temp_df.NumSamples.sum()), 2))
    print('Bias weighted average:', np.round((temp_df.Bias * temp_df.NumSamples).sum() / temp_df.NumSamples.sum(), 2))

### Grid cell LFMC estimation bias

In [8]:
def display_grid_bias(locations):
    print('Proportion of grid cells with under-estimated LFMC:', np.round(locations.Bias.lt(0).sum() / locations.shape[0], 2))
    print('Proportion of grid cells with abs(bias) < 5:', np.round(locations.Bias.between(-5, 5).sum() / locations.shape[0], 2))
    print('Proportion of grid cells with abs(bias) < 10:', np.round(locations.Bias.between(-10, 10).sum() / locations.shape[0], 2))
    print('Proportion of grid cells with abs(bias) > 20:', np.round(locations.Bias.abs().gt(20).sum() / locations.shape[0], 2))

### Generate sites tiff

In [9]:
def gen_locations_tiff(locations, output_file, bands, nodata_value, longitude='Longitude', latitude='Latitude'):
    x_coords = (int(np.floor(locations.index.get_level_values(longitude).min())), int(np.ceil(locations.index.get_level_values(longitude).max())))
    y_coords = (int(np.floor(locations.index.get_level_values(latitude).min())), int(np.ceil(locations.index.get_level_values(latitude).max())))
    x_size = (x_coords[1] - x_coords[0]) * 2 + 1
    y_size = (y_coords[1] - y_coords[0]) * 2 + 1
    multi_index = pd.MultiIndex.from_product([np.linspace(y_coords[0], y_coords[1], y_size), np.linspace(x_coords[0], x_coords[1], x_size)])

    pixel_size = 0.5
    transform = [x_coords[0], pixel_size, 0.0, y_coords[1], 0.0, -pixel_size]

    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4326)

    driver = gdal.GetDriverByName('GTiff')
    out_map_raster = driver.Create(output_file, x_size, y_size, len(bands), gdal.GDT_Float32)
    out_map_raster.SetGeoTransform(transform)
    out_map_raster.SetProjection(srs.ExportToWkt())

    for num, band in enumerate(bands, 1):
        band_data = locations[band].reindex(multi_index).unstack()[::-1]
        out_map_band = out_map_raster.GetRasterBand(num)
        out_map_band.SetNoDataValue(nodata_value)
        out_map_band.SetDescription(band)
        out_map_band.WriteArray(band_data.values)
        out_map_band.FlushCache()

    del out_map_raster

## Multi-tempCNN Within-site Scenario

In [10]:
print(f'Multi-tempCNN {SCENARIOS[0].capitalize()} Scenario')
print('==================================')

with open(os.path.join(MODELS_DIR[0], 'model_params.json'), 'r') as f:
    model_params = json.load(f)

predict = pd.read_csv(os.path.join(MODELS_DIR[0], ifile_names[0]), index_col=0)
samples = all_samples.reindex(predict.index)

locations = gen_gridded_data(predict, samples, model_params['targetColumn'])
print('\nResults summary')
print('---------------')
display_summary(locations)

print('\nResults for southern Texas')
print('--------------------------')
display_region(locations, (-100.0, -98.0), (26.0, 29.5))

print('\nResults for Rocky Mountains')
print('---------------------------')
display_region(locations, (-118.0, -113.0), (47.0, 50.0))

print('\nBias summary')
print('------------')
display_grid_bias(locations)

ofile = os.path.join(OUTPUT_DIR, ofile_names[0])
gen_locations_tiff(locations, ofile, BANDS, NODATA)

Multi-tempCNN Within-site Scenario

Results summary
---------------
Proportion of grid cells with RMSE < 20: 0.57 



,Bias,R,R2,RMSE,ubRMSE,NumSamples
min,-42.62,-0.42,-9.38,3.67,3.19,10.00
median,0.77,0.71,0.35,18.40,17.10,39.00
mean,0.14,0.64,0.17,20.57,18.69,51.01
max,35.54,0.98,0.95,79.25,73.92,528.00
std dev,9.53,0.27,0.92,10.26,9.40,48.77



Results for southern Texas
--------------------------
RMSE weighted average: 51.63
Bias weighted average: -19.69

Results for Rocky Mountains
---------------------------
RMSE weighted average: 38.13
Bias weighted average: -6.27

Bias summary
------------
Proportion of grid cells with under-estimated LFMC: 0.46
Proportion of grid cells with abs(bias) < 5: 0.51
Proportion of grid cells with abs(bias) < 10: 0.78
Proportion of grid cells with abs(bias) > 20: 0.05


## Multi-tempCNN Out-of-site Scenario

In [11]:
print(f'Multi-tempCNN {SCENARIOS[1].capitalize()} Scenario')
print('==================================')

with open(os.path.join(MODELS_DIR[1], 'model_params.json'), 'r') as f:
    model_params = json.load(f)

predict = pd.read_csv(os.path.join(MODELS_DIR[1], ifile_names[1]), index_col=0)

locations = gen_gridded_data(predict, all_samples, model_params['targetColumn'])
print('\nResults summary')
print('---------------')
display_summary(locations)

print('\nResults for southern Texas')
print('--------------------------')
display_region(locations, (-100.0, -98.0), (26.0, 29.5))

print('\nResults for Rocky Mountains')
print('---------------------------')
display_region(locations, (-118.0, -113.0), (47.0, 50.0))

print('\nBias summary')
print('------------')
display_grid_bias(locations)

ofile = os.path.join(OUTPUT_DIR, ofile_names[1])
gen_locations_tiff(locations, ofile, BANDS, NODATA)

Multi-tempCNN Out-of-site Scenario

Results summary
---------------
Proportion of grid cells with RMSE < 20: 0.38 



,Bias,R,R2,RMSE,ubRMSE,NumSamples
min,-53.11,-0.72,-64.24,4.80,4.53,10.00
median,1.79,0.63,0.15,22.94,19.91,95.00
mean,0.90,0.56,-0.34,24.88,21.52,141.04
max,59.94,0.98,0.94,97.56,92.12,1656.00
std dev,13.58,0.32,3.28,11.37,10.00,160.20



Results for southern Texas
--------------------------
RMSE weighted average: 58.68
Bias weighted average: -23.57

Results for Rocky Mountains
---------------------------
RMSE weighted average: 46.92
Bias weighted average: 1.07

Bias summary
------------
Proportion of grid cells with under-estimated LFMC: 0.41
Proportion of grid cells with abs(bias) < 5: 0.34
Proportion of grid cells with abs(bias) < 10: 0.61
Proportion of grid cells with abs(bias) > 20: 0.13


## Modis-tempCNN Within-site Scenario

In [12]:
print(f'Modis-tempCNN {SCENARIOS[0].capitalize()} Scenario')
print('==================================')

with open(os.path.join(MODELS_DIR[2], 'model_params.json'), 'r') as f:
    model_params = json.load(f)

predict = pd.read_csv(os.path.join(MODELS_DIR[2], ifile_names[2]), index_col=0)
samples = all_samples.reindex(predict.index)

locations = gen_gridded_data(predict, samples, model_params['targetColumn'])
print('\nResults summary')
print('---------------')
display_summary(locations)

print('\nResults for southern Texas')
print('--------------------------')
display_region(locations, (-100.0, -98.0), (26.0, 29.5))

print('\nResults for Rocky Mountains')
print('---------------------------')
display_region(locations, (-118.0, -113.0), (47.0, 50.0))

print('\nBias summary')
print('------------')
display_grid_bias(locations)

ofile = os.path.join(OUTPUT_DIR, ofile_names[2])
gen_locations_tiff(locations, ofile, BANDS, NODATA)

Modis-tempCNN Within-site Scenario

Results summary
---------------
Proportion of grid cells with RMSE < 20: 0.46 



,Bias,R,R2,RMSE,ubRMSE,NumSamples
min,-98.67,-0.64,-35.33,7.68,6.28,10.00
median,-2.93,0.63,0.18,21.05,18.59,39.00
mean,-4.13,0.56,-0.11,23.69,20.65,51.01
max,30.87,0.98,0.89,119.23,71.45,528.00
std dev,13.07,0.29,1.96,12.35,9.84,48.77



Results for southern Texas
--------------------------
RMSE weighted average: 89.33
Bias weighted average: -63.59

Results for Rocky Mountains
---------------------------
RMSE weighted average: 43.99
Bias weighted average: -8.44

Bias summary
------------
Proportion of grid cells with under-estimated LFMC: 0.63
Proportion of grid cells with abs(bias) < 5: 0.38
Proportion of grid cells with abs(bias) < 10: 0.64
Proportion of grid cells with abs(bias) > 20: 0.12


## Modis-tempCNN Out-of-site Scenario

In [13]:
print(f'Modis-tempCNN {SCENARIOS[1].capitalize()} Scenario')
print('==================================')

with open(os.path.join(MODELS_DIR[3], 'model_params.json'), 'r') as f:
    model_params = json.load(f)

predict = pd.read_csv(os.path.join(MODELS_DIR[3], ifile_names[3]), index_col=0)

locations = gen_gridded_data(predict, all_samples, model_params['targetColumn'])
print('\nResults summary')
print('---------------')
display_summary(locations)

print('\nResults for southern Texas')
print('--------------------------')
display_region(locations, (-100.0, -98.0), (26.0, 29.5))

print('\nResults for Rocky Mountains')
print('---------------------------')
display_region(locations, (-118.0, -113.0), (47.0, 50.0))

print('\nBias summary')
print('------------')
display_grid_bias(locations)

ofile = os.path.join(OUTPUT_DIR, ofile_names[3])
gen_locations_tiff(locations, ofile, BANDS, NODATA)

Modis-tempCNN Out-of-site Scenario

Results summary
---------------
Proportion of grid cells with RMSE < 20: 0.32 



,Bias,R,R2,RMSE,ubRMSE,NumSamples
min,-72.81,-0.73,-36.49,8.02,6.19,10.00
median,-1.76,0.52,0.05,24.12,21.76,95.00
mean,-3.34,0.49,-0.39,26.65,23.09,141.04
max,41.85,0.97,0.87,92.11,85.08,1656.00
std dev,14.66,0.32,2.27,12.60,10.29,160.20



Results for southern Texas
--------------------------
RMSE weighted average: 86.97
Bias weighted average: -64.15

Results for Rocky Mountains
---------------------------
RMSE weighted average: 49.88
Bias weighted average: -0.26

Bias summary
------------
Proportion of grid cells with under-estimated LFMC: 0.57
Proportion of grid cells with abs(bias) < 5: 0.37
Proportion of grid cells with abs(bias) < 10: 0.58
Proportion of grid cells with abs(bias) > 20: 0.15
